In [17]:
import numpy as np
import open3d as o3d
import os
import json
from scipy.spatial import cKDTree

def compute_metrics(gt_pcd, gen_pcd):
    """
    Computes various metrics to evaluate the accuracy of a generated point cloud against the ground truth.
    Handles cases where the point counts differ.
    
    Args:
        gt_points (numpy.ndarray): Ground truth points, shape (N1, 3).
        gen_points (numpy.ndarray): Generated points, shape (N2, 3).
        
    Returns:
        dict: A dictionary containing the computed metrics.
    """
    # Convert to numpy arrays
    gt_points = np.asarray(gt_pcd.points)
    gen_points = np.asarray(gen_pcd.points)

    # Compute pairwise distances using k-D tree for efficiency
    gt_kdtree = cKDTree(gt_points)
    gen_kdtree = cKDTree(gen_points)


    gt_set = set(map(tuple, gt_points))
    gen_set = set(map(tuple, gen_points))
    
    # For Chamfer Distance and Hausdorff Distance
    gen_to_gt_distances, _ = gen_kdtree.query(gt_points, k=1)
    gt_to_gen_distances, _ = gt_kdtree.query(gen_points, k=1)

    
    aabb = gt_pcd.get_axis_aligned_bounding_box()
    aabb_extent = aabb.get_extent()  # [dx, dy, dz]
    aabb_diagonal = np.linalg.norm(aabb_extent)  # Diagonal length
    
    # Metrics
    # Chamfer Distance (First Term)
    chamfer_distance = np.mean(gen_to_gt_distances ** 2) + np.mean(gt_to_gen_distances ** 2)
    
    # Hausdorff Distance
    hausdorff_distance = max(np.max(gen_to_gt_distances), np.max(gt_to_gen_distances))
    hausdorff_bb_ratio = hausdorff_distance / aabb_diagonal

    # MSE and RMSE
    mse = np.mean(gen_to_gt_distances ** 2)  # Only for nearest points
    rmse = np.sqrt(mse)
    
    # MAE
    mae = np.mean(gen_to_gt_distances)  # Absolute distance
    
    # Signal-to-Noise Ratio (SNR)
    signal_power = np.sum(np.linalg.norm(gt_points, axis=1) ** 2)
    noise_power = np.sum(gen_to_gt_distances ** 2)
    snr = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')

    # Exact matches
    exact_matches = gt_set & gen_set
    matched_count = len(exact_matches)
    
    # Missing and extra points
    missing_count = len(gt_set - gen_set)
    extra_count = len(gen_set - gt_set)
    
    # Completeness (Recall)
    completeness = matched_count / len(gt_set) if len(gt_set) > 0 else 1.0
    
    # Accuracy (Precision)
    accuracy = matched_count / len(gen_set) if len(gen_set) > 0 else 1.0
    
    # F1 Score
    if completeness + accuracy > 0:
        f1_score = 2 * (completeness * accuracy) / (completeness + accuracy)
    else:
        f1_score = 0.0
    
        
    return {
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "Chamfer Distance": chamfer_distance,
        "Hausdorff Distance": hausdorff_distance,
        "Bounding Box Diagonal": aabb_diagonal,
        "Hausdorff / BB Ratio": hausdorff_bb_ratio,
        "SNR": snr,
        "Exact Matches": matched_count,
        "Missing Points": missing_count,
        "Extra Points": extra_count,
        "Completeness/Recall": completeness,
        "Accuracy/Precision": accuracy,
        "F1 Score": f1_score,
        "Chamfer Distance": chamfer_distance
    }


def ply_metrics(gt_path, gen_dir, name, output_file):
    """
    Computes metrics for all .ply files in the given directories matching a specific naming pattern.
    
    Args:
        gt_dir (str): Directory containing the ground truth point clouds.
        gen_dir (str): Directory containing the generated point clouds.
        name (str): Base name for the point cloud files (e.g., "example").
        output_file (str): Path to the file where results will be saved.
        
    Returns:
        None
    """
    # Define the file patterns to process
    file_patterns = [f"{name}_density.ply", f"{name}_final.ply", f"{name}_floorseg.ply"]
    results = {}
    gt_pcd = o3d.io.read_point_cloud(gt_path)

    for file_pattern in file_patterns:
        gen_path = os.path.join(gen_dir, file_pattern)
        print()
        print(gen_path)
        
        # Check if both files exist
        if not os.path.exists(gen_path):
            print(f"Skipping {file_pattern} as gen file is missing.")
            continue
        
        # Load point clouds
        gen_pcd = o3d.io.read_point_cloud(gen_path)
        
        # Convert to numpy arrays
        
        # Compute metrics
        metrics = compute_metrics(gt_pcd, gen_pcd)
        results[file_pattern] = metrics
        print(f"Metrics for {file_pattern}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value}")
    
    # Save results to a file
    with open(output_file, "w") as f:
        json.dump(results, f, indent=4)
    print(f"Results saved to {output_file}")
    return results

In [1]:
import numpy as np

# Define the metrics for each scene
metrics = {
    "kitchen_dense.ply": {
        "Chamfer Distance": 0.28121,
        "Hausdorff Distance": 1.306,
        "HD/BB": 0.472,
        "Exact Matches": 37630,
        "Missing Points": 8,
        "Extra Points": 97984,
        "Completeness/Recall": 0.999,
        "Accuracy/Precision": 0.277,
        "F1 Score": 0.434
    },
    "kitchen_floorseg.ply": {
        "Chamfer Distance": 0.00920,
        "Hausdorff Distance": 1.283,
        "HD/BB": 0.463,
        "Exact Matches": 35423,
        "Missing Points": 2215,
        "Extra Points": 1042,
        "Completeness/Recall": 0.941,
        "Accuracy/Precision": 0.971,
        "F1 Score": 0.956
    },
    "kitchen_final.ply": {
        "Chamfer Distance": 0.00038,
        "Hausdorff Distance": 0.445,
        "HD/BB": 0.161,
        "Exact Matches": 35423,
        "Missing Points": 2215,
        "Extra Points": 943,
        "Completeness/Recall": 0.941,
        "Accuracy/Precision": 0.974,
        "F1 Score": 0.957
    },
    "bicycle_dense.ply": {
        "Chamfer Distance": 0.43860,
        "Hausdorff Distance": 2.329,
        "HD/BB": 0.612,
        "Exact Matches": 6334,
        "Missing Points": 0,
        "Extra Points": 10643,
        "Completeness/Recall": 1.000,
        "Accuracy/Precision": 0.373,
        "F1 Score": 0.543
    },
    "bicycle_floorseg.ply": {
        "Chamfer Distance": 0.01661,
        "Hausdorff Distance": 2.328,
        "HD/BB": 0.611,
        "Exact Matches": 6272,
        "Missing Points": 62,
        "Extra Points": 475,
        "Completeness/Recall": 0.990,
        "Accuracy/Precision": 0.930,
        "F1 Score": 0.959
    },
    "bicycle_final.ply": {
        "Chamfer Distance": 0.00264,
        "Hausdorff Distance": 0.478,
        "HD/BB": 0.125,
        "Exact Matches": 6272,
        "Missing Points": 62,
        "Extra Points": 443,
        "Completeness/Recall": 0.990,
        "Accuracy/Precision": 0.934,
        "F1 Score": 0.961
    },
    "bonsai_dense.ply": {
        "Chamfer Distance": 0.22608,
        "Hausdorff Distance": 1.906,
        "HD/BB": 0.641,
        "Exact Matches": 25220,
        "Missing Points": 0,
        "Extra Points": 101654,
        "Completeness/Recall": 1.000,
        "Accuracy/Precision": 0.199,
        "F1 Score": 0.332
    },
    "bonsai_floorseg.ply": {
        "Chamfer Distance": 0.00127,
        "Hausdorff Distance": 1.351,
        "HD/BB": 0.455,
        "Exact Matches": 24710,
        "Missing Points": 510,
        "Extra Points": 216,
        "Completeness/Recall": 0.980,
        "Accuracy/Precision": 0.991,
        "F1 Score": 0.986
    },
    "bonsai_final.ply": {
        "Chamfer Distance": 0.00031,
        "Hausdorff Distance": 0.636,
        "HD/BB": 0.214,
        "Exact Matches": 24710,
        "Missing Points": 510,
        "Extra Points": 181,
        "Completeness/Recall": 0.980,
        "Accuracy/Precision": 0.993,
        "F1 Score": 0.986
    },
    "garden_dense.ply": {
        "Chamfer Distance": 0.99917,
        "Hausdorff Distance": 2.349,
        "HD/BB": 0.479,
        "Exact Matches": 14055,
        "Missing Points": 0,
        "Extra Points": 37227,
        "Completeness/Recall": 1.000,
        "Accuracy/Precision": 0.274,
        "F1 Score": 0.430
    },
    "garden_floorseg.ply": {
        "Chamfer Distance": 0.64211,
        "Hausdorff Distance": 2.349,
        "HD/BB": 0.479,
        "Exact Matches": 13175,
        "Missing Points": 880,
        "Extra Points": 3452,
        "Completeness/Recall": 0.937,
        "Accuracy/Precision": 0.792,
        "F1 Score": 0.859
    },
    "garden_final.ply": {
        "Chamfer Distance": 0.00101,
        "Hausdorff Distance": 0.697,
        "HD/BB": 0.142,
        "Exact Matches": 13171,
        "Missing Points": 884,
        "Extra Points": 142,
        "Completeness/Recall": 0.937,
        "Accuracy/Precision": 0.989,
        "F1 Score": 0.963
    }
}


# Separate the file types for averaging
file_types = ["dense", "final", "floorseg"]
scene_names = ["bonsai", "bicycle", "garden", "kitchen"]

# Initialize a dictionary to store averages
averages = {ft: {} for ft in file_types}

# Iterate through each file type and calculate averages
for ft in file_types:
    # Collect metrics for the specific file type
    ft_metrics = {key: metrics[key] for key in metrics if ft in key}
    
    # Calculate average for each metric
    avg_metrics = {}
    for metric_key in next(iter(ft_metrics.values())).keys():  # Get metric keys from the first entry
        avg_metrics[metric_key] = np.mean([ft_metrics[file][metric_key] for file in ft_metrics])
    
    averages[ft] = avg_metrics

# Print average metrics for each file type
for ft, avg_metrics in averages.items():
    print(f"Average metrics for {ft}.ply:")
    for metric, value in avg_metrics.items():
        print(f"  {metric}: {value:.5f}")
    print()

# Write average metrics to a file
with open("metrics.txt", "w") as file:
    for ft, avg_metrics in averages.items():
        file.write(f"Average metrics for {ft}.ply:\n")
        for metric, value in avg_metrics.items():
            file.write(f"  {metric}: {value:.5f}\n")
        file.write("\n")


Average metrics for dense.ply:
  Chamfer Distance: 0.48627
  Hausdorff Distance: 1.97250
  HD/BB: 0.55100
  Exact Matches: 20809.75000
  Missing Points: 2.00000
  Extra Points: 61877.00000
  Completeness/Recall: 0.99975
  Accuracy/Precision: 0.28075
  F1 Score: 0.43475

Average metrics for final.ply:
  Chamfer Distance: 0.00109
  Hausdorff Distance: 0.56400
  HD/BB: 0.16050
  Exact Matches: 19894.00000
  Missing Points: 917.75000
  Extra Points: 427.25000
  Completeness/Recall: 0.96200
  Accuracy/Precision: 0.97250
  F1 Score: 0.96675

Average metrics for floorseg.ply:
  Chamfer Distance: 0.16730
  Hausdorff Distance: 1.82775
  HD/BB: 0.50200
  Exact Matches: 19895.00000
  Missing Points: 916.75000
  Extra Points: 1296.25000
  Completeness/Recall: 0.96200
  Accuracy/Precision: 0.92100
  F1 Score: 0.94000



In [18]:
bicycle_metrics = ply_metrics("data/points_bicycle_gt.ply", "output/bicycle/cluster", "bicycle", "output/bicycle/cluster/metrics.json")



output/bicycle/cluster\bicycle_density.ply
Metrics for bicycle_density.ply:
  MSE: 0.0
  RMSE: 0.0
  MAE: 0.0
  Chamfer Distance: 0.297468168421502
  Hausdorff Distance: 1.8987165015454919
  Bounding Box Diagonal: 3.8078062307677456
  Hausdorff / BB Ratio: 0.49863789974487877
  SNR: inf
  Exact Matches: 6334
  Missing Points: 0
  Extra Points: 9256
  Completeness/Recall: 1.0
  Accuracy/Precision: 0.4062860808210391
  F1 Score: 0.5778142674694399

output/bicycle/cluster\bicycle_final.ply
Metrics for bicycle_final.ply:
  MSE: 1.4679513616464817e-05
  RMSE: 0.0038313853390731685
  MAE: 0.0002952980516493955
  Chamfer Distance: 0.0035690604638450216
  Hausdorff Distance: 0.430407705370072
  Bounding Box Diagonal: 3.8078062307677456
  Hausdorff / BB Ratio: 0.11303298521135395
  SNR: 50.0303663037498
  Exact Matches: 6281
  Missing Points: 53
  Extra Points: 506
  Completeness/Recall: 0.9916324597410799
  Accuracy/Precision: 0.9254457050243112
  F1 Score: 0.9573965398978738

output/bicycle/

MSE: 1.2573330356184421e-05
RMSE: 0.003545889219389746
MAE: 0.00021460280284468124
Chamfer Distance: 0.004557185225284462
Hausdorff Distance: 0.4778152120945047
Maximum Distance GT: 0.3307482331216485
Maximum Distance Gen: 0.3893983222353729
SNR: 50.70297969988866
Exact Matches: 6308
Missing Points: 26
Extra Points: 689
Completeness: 0.9958951689295864
Accuracy: 0.9015292268114906

In [69]:
bonsai_metrics = ply_metrics("data/points_bonsai_gt.ply", "output/bonsai/cluster", "bonsai", "output/bonsai/cluster/metrics.json")



output/bonsai/cluster\bonsai_density.ply
Metrics for bonsai_density.ply:
  MSE: 0.0
  RMSE: 0.0
  MAE: 0.0
  Chamfer Distance: 0.22602038574583713
  Hausdorff Distance: 1.9056247417840848
  Bounding Box Diagonal: 2.9727479402394965
  Hausdorff / BB Ratio: 0.6410313891700351
  SNR: inf
  Exact Matches: 25220
  Missing Points: 0
  Extra Points: 101650
  Completeness/Recall: 1.0
  Accuracy/Precision: 0.19878615906045558
  F1 Score: 0.3316457360773226

output/bonsai/cluster\bonsai_final.ply
Metrics for bonsai_final.ply:
  MSE: 0.0002942565909399683
  RMSE: 0.017153908911381344
  MAE: 0.0029390689556676817
  Chamfer Distance: 0.00035447929147648313
  Hausdorff Distance: 0.2453098323343594
  Bounding Box Diagonal: 2.9727479402394965
  Hausdorff / BB Ratio: 0.08251955337814354
  SNR: 40.81516682514785
  Exact Matches: 24363
  Missing Points: 857
  Extra Points: 128
  Completeness/Recall: 0.9660190325138779
  Accuracy/Precision: 0.994773590298477
  F1 Score: 0.9801854720283237

output/bonsai/

MSE: 0.0001709805336266227
RMSE: 0.013075952494048864
MAE: 0.001834674022331613
Chamfer Distance: 0.00023835957867605645
Hausdorff Distance: 0.2453098323343594
SNR: 43.172962135626975
Exact Matches: 24531
Missing Points: 689
Extra Points: 135
Completeness: 0.972680412371134
Accuracy: 0.9945268791048407

In [ ]:
kitchen_metrics = ply_metrics("data/points_kitchen_gt.ply", "output/kitchen/cluster", "kitchen", "output/kitchen/cluster/metrics.json")



output/kitchen/cluster\kitchen_density.ply
Metrics for kitchen_density.ply:
  MSE: 4.04493098696447e-08
  RMSE: 0.00020112013790181404
  MAE: 2.8520026236961695e-06
  Chamfer Distance: 0.28120830778752126
  Hausdorff Distance: 1.3058407282629578
  Bounding Box Diagonal: 2.7679881761429908
  Hausdorff / BB Ratio: 0.47176528408533913
  SNR: 80.43973653590727
  Exact Matches: 37630
  Missing Points: 8
  Extra Points: 97984
  Completeness/Recall: 0.9997874488548807
  Accuracy/Precision: 0.27747872638518145
  F1 Score: 0.4343961397271027

output/kitchen/cluster\kitchen_final.ply
Metrics for kitchen_final.ply:
  MSE: 0.00010741822662737857
  RMSE: 0.010364276464248654
  MAE: 0.002171543334584353
  Chamfer Distance: 0.00038061407328055153
  Hausdorff Distance: 0.4456950648320692
  Bounding Box Diagonal: 2.7679881761429908
  Hausdorff / BB Ratio: 0.16101769099791313
  SNR: 46.198067915287254
  Exact Matches: 35423
  Missing Points: 2215
  Extra Points: 943
  Completeness/Recall: 0.94114990169

output/kitchen/cluster\kitchen_density.ply
Metrics for kitchen_density.ply:
  MSE: 0.0
  RMSE: 0.0
  MAE: 0.0
  Chamfer Distance: 0.3150447259020694
  Hausdorff Distance: 1.3619621770644579
  Bounding Box Diagonal: 2.7679881761429908
  Hausdorff / BB Ratio: 0.49204046057821765
  SNR: inf
  Exact Matches: 37638
  Missing Points: 0
  Extra Points: 104054
  Completeness/Recall: 1.0
  Accuracy/Precision: 0.2656324986590633
  F1 Score: 0.41976244911615457

output/kitchen/cluster\kitchen_final.ply
Metrics for new_kitchen_final.ply:
  MSE: 6.001387279951112e-05
  RMSE: 0.007746862126016644
  MAE: 0.0013294388963682243
  Chamfer Distance: 0.000788100648507937
  Hausdorff Distance: 0.6740201589955116
  Bounding Box Diagonal: 2.7679881761429908
  Hausdorff / BB Ratio: 0.2435054328644981
  SNR: 48.72633116390997
  Exact Matches: 36151
  Missing Points: 1487
  Extra Points: 1114
  Completeness/Recall: 0.9604920559009512
  Accuracy/Precision: 0.9701059975848652
  F1 Score: 0.9652750891152557

output/kitchen/cluster\kitchen_floorseg.ply
Metrics for kitchen_floorseg.ply:
  MSE: 0.0007681347634677821
  RMSE: 0.027715244243336233
  MAE: 0.008878417324621793
  Chamfer Distance: 0.0021158302814730683
  Hausdorff Distance: 1.3001832493126975
  Bounding Box Diagonal: 2.7679881761429908
  Hausdorff / BB Ratio: 0.4697213884505884
  SNR: 37.65447349422094
  Exact Matches: 32916
  Missing Points: 4722
  Extra Points: 837
  Completeness/Recall: 0.8745416865933365
  Accuracy/Precision: 0.9752022042485112
  F1 Score: 0.9221330419800815
Results saved to output/kitchen/cluster/metrics.json

In [71]:
garden_metrics = ply_metrics("data/points_garden_gt.ply", "output/garden/cluster", "garden", "output/garden/cluster/metrics.json")



output/garden/cluster\garden_density.ply
Metrics for garden_density.ply:
  MSE: 0.0
  RMSE: 0.0
  MAE: 0.0
  Chamfer Distance: 2.5403588049146486
  Hausdorff Distance: 4.34449096644896
  Bounding Box Diagonal: 4.9021048569431915
  Hausdorff / BB Ratio: 0.8862501095413241
  SNR: inf
  Exact Matches: 14055
  Missing Points: 0
  Extra Points: 51018
  Completeness/Recall: 1.0
  Accuracy/Precision: 0.21598819787008436
  F1 Score: 0.3552471944191689

output/garden/cluster\garden_final.ply
Metrics for garden_final.ply:
  MSE: 0.0012241690711216346
  RMSE: 0.03498812757381616
  MAE: 0.010007045991085348
  Chamfer Distance: 0.0013157157168922852
  Hausdorff Distance: 0.3944644905621165
  Bounding Box Diagonal: 4.9021048569431915
  Hausdorff / BB Ratio: 0.08046839104296373
  SNR: 36.222138912315366
  Exact Matches: 12720
  Missing Points: 1335
  Extra Points: 37
  Completeness/Recall: 0.9050160085378869
  Accuracy/Precision: 0.9970996315748216
  F1 Score: 0.9488288825898852

output/garden/clust

MSE: 0.0011595296083539836
RMSE: 0.034051866444498806
MAE: 0.009683864530969196
Chamfer Distance: 0.0012840190732325805
Hausdorff Distance: 0.5481564818660584
SNR: 36.45773451258344
Exact Matches: 12733
Missing Points: 1322
Extra Points: 38
Completeness: 0.9059409462824618
Accuracy: 0.9970245086524157